In [1]:

from keras.utils import to_categorical
from keras_preprocessing.image import load_img
# from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
import PIL

In [2]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [4]:

train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:

print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [6]:

test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
print(test)
print(test['image'])

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         images/test\angry\10052.jpg
1         images/test\angry\10065.jpg
2         images/test\angry\10079.jpg
3         images/test\angry\10095.jpg
4         images/test\angry\10121.jpg
                    ...              
7061    images/test\surprise\9806.jpg
7062    images/test\surprise\9830.jpg
7063    images/test\surprise\9853.jpg
7064    images/test\surprise\9878.jpg
7065     images/test\surprise\993.jpg
Name:

In [8]:
import PIL.Image

from tqdm.notebook import tqdm

In [11]:
from keras_preprocessing.image import load_img
import numpy as np
from tqdm.notebook import tqdm

def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale', target_size=(48, 48))  # ✅ resize and grayscale
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)  # now this will work
    return features


In [12]:
t = train['image']
train_features = extract_features(t) 

  0%|          | 0/28821 [00:00<?, ?it/s]

In [13]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [16]:
x_train = train_features/255.0
x_test = test_features/255.0

In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [19]:

y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [20]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [23]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 246s 1s/step - accuracy: 0.2351 - loss: 1.8400 - val_accuracy: 0.2604 - val_loss: 1.8129
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 241s 1s/step - accuracy: 0.2503 - loss: 1.7964 - val_accuracy: 0.3055 - val_loss: 1.6917
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 242s 1s/step - accuracy: 0.3030 - loss: 1.7002 - val_accuracy: 0.3866 - val_loss: 1.5495
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 247s 1s/step - accuracy: 0.3860 - loss: 1.5571 - val_accuracy: 0.4802 - val_loss: 1.3862
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 253s 1s/step - accuracy: 0.4388 - loss: 1.4497 - val_accuracy: 0.4856 - val_loss: 1.3358
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 241s 1s/step - accuracy: 0.4604 - loss: 1.3984 - val_accuracy: 0.5085 - val_loss: 1.2962
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 234s 1s/step - accuracy: 0.4813 - loss: 1.3605 - val_accuracy: 0.5241 - val_loss: 1.2544
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 245s 1s/step - accuracy: 0.4862 - loss: 1.3316 - 

KeyboardInterrupt: 

In [27]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_cb = ModelCheckpoint(
    filepath='emotion_model_checkpoint.h5',
    save_best_only=False,          # You can set True to save only best val_accuracy
    save_weights_only=False,       # Save the entire model, not just weights
    monitor='val_accuracy',        # Optional
    verbose=1
)


In [28]:
model.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_test, y_test),
    callbacks=[checkpoint_cb],
    initial_epoch=47              # 🔥 RESUME FROM HERE later
)


Epoch 48/100
  8/226 ━━━━━━━━━━━━━━━━━━━━ 4:11 1s/step - accuracy: 0.6560 - loss: 0.9653

KeyboardInterrupt: 